In [1]:
import pandas as pd
import math
import random

In [2]:
df= pd.read_csv("./TennisDataset.csv")
df.head()

,Day,Outlook,Temperature,Humidity,Wind,PlayTennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [3]:
target_attribute= "PlayTennis"
attributes= {"Outlook","Temperature","Humidity","Wind"}

In [4]:
Dd=df[1:2]
print(Dd)
print((Dd[target_attribute]==Dd[target_attribute][1]).all())
print(df.loc[0]["Outlook"])

  Day Outlook Temperature Humidity    Wind PlayTennis
1  D2   Sunny         Hot     High  Strong         No
True
Sunny


In [5]:
len(df[target_attribute]==df[target_attribute][0])
df[df[target_attribute]=="Yes"]

,Day,Outlook,Temperature,Humidity,Wind,PlayTennis
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
6,D7,Overcast,Cool,Normal,Strong,Yes
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes
10,D11,Sunny,Mild,Normal,Strong,Yes
11,D12,Overcast,Mild,High,Strong,Yes
12,D13,Overcast,Hot,Normal,Weak,Yes


In [6]:
class leaf_node:
    def __init__(self,val=None):
        self.value= val
class decision_node:
    def __init__(self):
        self.splitting_attribute= None
        self.children={} # dictionary

In [7]:
def entropy(D,target_attribute):
    e=0
    for c in set(D[target_attribute]):
        Pc= len(D[D[target_attribute]==c]) / len(D)
        e -= Pc * math.log2(Pc)
    return e

def calc_gain(D,target_attribute,attribute):
    print(attribute,':',len(D),end=' = ')
    gain= entropy(D,target_attribute)
    for c in set(D[attribute]):
        print(c, len(D[D[attribute]==c]), end=' + ')
        gain = gain - len(D[D[attribute]==c])/len(D)*entropy(D[D[attribute]==c] , target_attribute)
    print()
    print("gain =",gain)
    return gain

def common_value(D, target_attribute):
    a=None
    for i in D[target_attribute]:
        if a is None:
            a=i
        elif i!= a:
            return None
    return a

print(entropy(df[0:1],target_attribute))
print('allsame=',common_value(df[2:5],target_attribute))


0.0
allsame= Yes


In [8]:
def ID3(D,target_attribute,attributes):
    print("Entered", len(D))
    # if all target labels are equal
    a= common_value(D, target_attribute)
    if  a is not None:
        print("Leaf Value: ", a)
        return leaf_node(val= a)

    print("Choosing an attribute.")
    maxx=0
    splitting_attribute = None
    for a in attributes:
        gain= calc_gain(D, target_attribute, a)
        if gain>maxx:
            splitting_attribute= a
            maxx=gain
    print("splitting attrbt: ", splitting_attribute)

    maxx=0
    most_common_element= None
    for val in set(D[target_attribute]):
        if len(D[D[target_attribute]==val])>maxx:
            most_common_element= val
            maxx= len(D[D[target_attribute]==val])
    
    # in case of no attributes remaining, or none that provide positive information gain
    if splitting_attribute is None:
        return leaf_node(val=most_common_element)
    
    root = decision_node()
    root.splitting_attribute = splitting_attribute
    for c in set(D[splitting_attribute]):
        print("Class",c,len(D[D[splitting_attribute]==c]))
        root.children[c]=ID3(D[D[splitting_attribute]==c], target_attribute, attributes-{splitting_attribute})
    root.children["Other"]= leaf_node(most_common_element)

    return root



In [9]:
def predict(decision_tree,x):
    a=decision_tree
    while isinstance(a,decision_node):
        print(a.splitting_attribute, end='')
        print( ":", x[a.splitting_attribute])
        if x[a.splitting_attribute] in a.children:
            a= a.children[x[a.splitting_attribute]]
        else:
            a= a.children["Other"]
    return a.value

In [10]:
tr= []
te= []
for i in range(len(df)):
    if random.random()<0.8:
        tr.append(True)
        te.append(False)
    else:
        tr.append(False)
        te.append(True)
print(tr)
training_dataset= df[tr]
testing_dataset= df[te]
print("training:\n",training_dataset)
print("testing\n",testing_dataset)

[True, True, True, False, True, True, False, False, True, True, True, False, False, False]
training:
     Day   Outlook Temperature Humidity    Wind PlayTennis
0    D1     Sunny         Hot     High    Weak         No
1    D2     Sunny         Hot     High  Strong         No
2    D3  Overcast         Hot     High    Weak        Yes
4    D5      Rain        Cool   Normal    Weak        Yes
5    D6      Rain        Cool   Normal  Strong         No
8    D9     Sunny        Cool   Normal    Weak        Yes
9   D10      Rain        Mild   Normal    Weak        Yes
10  D11     Sunny        Mild   Normal  Strong        Yes
testing
     Day   Outlook Temperature Humidity    Wind PlayTennis
3    D4      Rain        Mild     High    Weak        Yes
6    D7  Overcast        Cool   Normal  Strong        Yes
7    D8     Sunny        Mild     High    Weak         No
11  D12  Overcast        Mild     High  Strong        Yes
12  D13  Overcast         Hot   Normal    Weak        Yes
13  D14      Rain  

In [11]:
decision_tree= ID3(training_dataset, target_attribute , attributes)

Entered 8
Choosing an attribute.
Wind : 8 = Strong 3 + Weak 5 + 
gain = 0.15886800584993
Humidity : 8 = High 3 + Normal 5 + 
gain = 0.15886800584993
Outlook : 8 = Sunny 4 + Rain 3 + Overcast 1 + 
gain = 0.11007306515453147
Temperature : 8 = Mild 2 + Cool 3 + Hot 3 + 
gain = 0.26571212738409783
splitting attrbt:  Temperature
Class Mild 2
Entered 2
Leaf Value:  Yes
Class Cool 3
Entered 3
Choosing an attribute.
Wind : 3 = Strong 1 + Weak 2 + 
gain = 0.9182958340544896
Humidity : 3 = Normal 3 + 
gain = 0.0
Outlook : 3 = Sunny 1 + Rain 2 + 
gain = 0.2516291673878229
splitting attrbt:  Wind
Class Strong 1
Entered 1
Leaf Value:  No
Class Weak 2
Entered 2
Leaf Value:  Yes
Class Hot 3
Entered 3
Choosing an attribute.
Wind : 3 = Strong 1 + Weak 2 + 
gain = 0.2516291673878229
Humidity : 3 = High 3 + 
gain = 0.0
Outlook : 3 = Sunny 2 + Overcast 1 + 
gain = 0.9182958340544896
splitting attrbt:  Outlook
Class Sunny 2
Entered 2
Leaf Value:  No
Class Overcast 1
Entered 1
Leaf Value:  Yes


In [12]:
# testing the decision tree
count=0
for i in testing_dataset.index:
    x= testing_dataset.loc[i]
    p=predict(decision_tree,x)
    print(x[target_attribute],"Prediction:",p)
    if p== x[target_attribute]:
        count+=1
print(f"Accuracy: {count}/{len(testing_dataset)}")

Temperature: Mild
Yes Prediction: Yes
Temperature: Cool
Wind: Strong
Yes Prediction: No
Temperature: Mild
No Prediction: Yes
Temperature: Mild
Yes Prediction: Yes
Temperature: Hot
Outlook: Overcast
Yes Prediction: Yes
Temperature: Mild
No Prediction: Yes
Accuracy: 3/6
